In [1]:
!pip install pycaret pandas shap


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from pycaret.regression import *
from datetime import timedelta
import numpy as np


# โหลดไฟล์ CSV
file_path = "D:/Term_Project/pm_2.5/cleaned_data (2)-7days.csv"
df = pd.read_csv(file_path)

# แปลง datetime เป็นชนิด datetime
df["datetime"] = pd.to_datetime(df["datetime"], dayfirst=True, errors="coerce")

# แปลง 'pm_2_5' เป็นตัวเลข
df["pm_2_5"] = pd.to_numeric(df["pm_2_5"], errors="coerce")

# สร้างสำเนา DataFrame เพื่อใช้ทำความสะอาดข้อมูล
df_cleaned = df.copy()

# ฟังก์ชันสำหรับจัดการ outlier และ NaN ด้วย Hybrid Imputation
def hybrid_imputation(series):
    # คำนวณ IQR
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    # กำหนดขอบเขตของ outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # แทนที่ outlier ด้วยค่ามัธยฐาน
    median_value = series.median()
    series = series.where((series >= lower_bound) & (series <= upper_bound), median_value)

    # เติมค่าที่หายไปด้วยค่ามัธยฐาน
    series = series.fillna(median_value)

    return series

# สร้าง Lag Features
lags = [1, 3, 6, 12, 24, 48, 168]  # Lag ที่ต้องการ
for lag in lags:
    df_cleaned[f"pm_2_5_lag_{lag}"] = df_cleaned["pm_2_5"].shift(lag)

# ใช้ Hybrid Imputation สำหรับคอลัมน์ 'pm_2_5' และ Lag Features
for col in ["pm_2_5"] + [f"pm_2_5_lag_{lag}" for lag in lags]:
    df_cleaned[col] = hybrid_imputation(df_cleaned[col])

# เพิ่มฟีเจอร์เชิงเวลา
df_cleaned["hour"] = df_cleaned["datetime"].dt.hour
df_cleaned["dayofweek"] = df_cleaned["datetime"].dt.dayofweek
df_cleaned["month"] = df_cleaned["datetime"].dt.month
df_cleaned["season"] = (df_cleaned["month"] % 12 + 3) // 3  # 1=Winter, 2=Spring, 3=Summer, 4=Fall
df_cleaned["is_daytime"] = df_cleaned["hour"].apply(lambda x: 1 if 6 <= x <= 18 else 0)  # กลางวัน=1, กลางคืน=0

# ตั้งค่า PyCaret
exp = setup(
    df_cleaned, 
    target="pm_2_5", 
    train_size=0.8, 
    session_id=42, 
    normalize=True, 
    feature_selection=True, 
    fold=5,  # ใช้ 5-fold Cross-Validation
)

# สร้างและเปรียบเทียบหลายโมเดล
best_model = compare_models()

# ปรับ Hyperparameters ของโมเดลที่ดีที่สุด
tuned_model = tune_model(best_model, optimize="RMSE", n_iter=50)

# ---- สร้างข้อมูลสำหรับพยากรณ์ล่วงหน้า 7 วัน (168 ชั่วโมง) ----
last_date = df_cleaned["datetime"].max()
future_dates = [last_date + timedelta(hours=i) for i in range(1, 169)]

# ใช้ค่าเฉลี่ยล่าสุดของความชื้นและอุณหภูมิ
latest_humidity = df_cleaned["humidity"].iloc[-24:].mean()
latest_temperature = df_cleaned["temperature"].iloc[-24:].mean()
latest_pm25 = df_cleaned["pm_2_5"].iloc[-1]

# สร้าง DataFrame สำหรับข้อมูลอนาคต
future_data = pd.DataFrame({
    "datetime": future_dates,
    "humidity": [latest_humidity] * len(future_dates),
    "temperature": [latest_temperature] * len(future_dates),
    **{f"pm_2_5_lag_{lag}": [latest_pm25] * len(future_dates) for lag in lags}
})

# เพิ่มฟีเจอร์เชิงเวลา
future_data["hour"] = future_data["datetime"].dt.hour
future_data["dayofweek"] = future_data["datetime"].dt.dayofweek
future_data["month"] = future_data["datetime"].dt.month
future_data["season"] = (future_data["month"] % 12 + 3) // 3
future_data["is_daytime"] = future_data["hour"].apply(lambda x: 1 if 6 <= x <= 18 else 0)

# Recursive forecasting
for i in range(len(future_data)):
    pred = predict_model(tuned_model, data=future_data.iloc[i:i+1])
    predicted_value = pred.iloc[0]["prediction_label"]
    
    # อัปเดต Lag Features สำหรับแถวถัดไป
    for lag in lags:
        if i + lag < len(future_data):
            future_data.at[i + lag, f"pm_2_5_lag_{lag}"] = predicted_value

# ทำนายค่า PM 2.5
predictions = predict_model(tuned_model, data=future_data)

# หาชื่อคอลัมน์การทำนายที่ถูกต้อง
prediction_column = [col for col in predictions.columns if "predict" in col.lower()]
if prediction_column:
    prediction_column = prediction_column[0]
    print(predictions[["datetime", prediction_column]])
else:
    print("ไม่พบคอลัมน์การทำนาย ลองแสดงผลทั้งหมด:")
    print(predictions.head())

# บันทึกผลลัพธ์เป็นไฟล์ CSV
# predictions.to_csv("D:/Term_Project/pm_2.5/predicted_pm25.csv", index=False)
# print("บันทึกผลลัพธ์การพยากรณ์สำเร็จ!")

,Description,Value
0,Session id,42
1,Target,pm_2_5
2,Target type,Regression
3,Original data shape,"(34465, 16)"
4,Transformed data shape,"(34465, 4)"
5,Transformed train set shape,"(27572, 4)"
6,Transformed test set shape,"(6893, 4)"
7,Numeric features,14
8,Date features,1
9,Preprocess,True


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,21:11:14
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from datetime import timedelta
import numpy as np

# โหลดไฟล์ CSV
file_path = "D:/Term_Project/pm_2.5/cleaned_data (2)-7days.csv"
df = pd.read_csv(file_path)

# แปลง datetime เป็นชนิด datetime
df["datetime"] = pd.to_datetime(df["datetime"], dayfirst=True, errors="coerce")

# แปลง 'pm_2_5' เป็นตัวเลข
df["pm_2_5"] = pd.to_numeric(df["pm_2_5"], errors="coerce")

# สร้างสำเนา DataFrame เพื่อใช้ทำความสะอาดข้อมูล
df_cleaned = df.copy()

# ฟังก์ชันสำหรับจัดการ outlier และ NaN ด้วย Hybrid Imputation
def hybrid_imputation(series):
    # คำนวณ IQR
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    # กำหนดขอบเขตของ outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # แทนที่ outlier ด้วยค่ามัธยฐาน
    median_value = series.median()
    series = series.where((series >= lower_bound) & (series <= upper_bound), median_value)

    # เติมค่าที่หายไปด้วยค่ามัธยฐาน
    series = series.fillna(median_value)

    return series

# สร้าง Lag Features
lags = [1, 3, 6, 12, 24, 48, 168]  # Lag ที่ต้องการ
for lag in lags:
    df_cleaned[f"pm_2_5_lag_{lag}"] = df_cleaned["pm_2_5"].shift(lag)

# ใช้ Hybrid Imputation สำหรับคอลัมน์ 'pm_2_5' และ Lag Features
for col in ["pm_2_5"] + [f"pm_2_5_lag_{lag}" for lag in lags]:
    df_cleaned[col] = hybrid_imputation(df_cleaned[col])

# เพิ่มฟีเจอร์เชิงเวลา
df_cleaned["hour"] = df_cleaned["datetime"].dt.hour
df_cleaned["dayofweek"] = df_cleaned["datetime"].dt.dayofweek
df_cleaned["month"] = df_cleaned["datetime"].dt.month
df_cleaned["season"] = (df_cleaned["month"] % 12 + 3) // 3  # 1=Winter, 2=Spring, 3=Summer, 4=Fall
df_cleaned["is_daytime"] = df_cleaned["hour"].apply(lambda x: 1 if 6 <= x <= 18 else 0)  # กลางวัน=1, กลางคืน=0

# ลบค่า NaN ที่เกิดจากการสร้าง Lag Features
df_cleaned.dropna(inplace=True)

# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
X = df_cleaned.drop(columns=["pm_2_5", "datetime"])  # ลบคอลัมน์ที่ไม่ใช้
y = df_cleaned["pm_2_5"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้างและฝึกโมเดล Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# ทำนายบนชุดทดสอบ
y_pred = rf_model.predict(X_test)

# ประเมินโมเดล
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

# ---- สร้างข้อมูลสำหรับพยากรณ์ล่วงหน้า 7 วัน (168 ชั่วโมง) ----
last_date = df_cleaned["datetime"].max()
future_dates = [last_date + timedelta(hours=i) for i in range(1, 169)]

# ใช้ค่าเฉลี่ยล่าสุดของความชื้นและอุณหภูมิ
latest_humidity = df_cleaned["humidity"].iloc[-24:].mean()
latest_temperature = df_cleaned["temperature"].iloc[-24:].mean()
latest_pm25 = df_cleaned["pm_2_5"].iloc[-1]

# สร้าง DataFrame สำหรับข้อมูลอนาคต
future_data = pd.DataFrame({
    "datetime": future_dates,
    "humidity": [latest_humidity] * len(future_dates),
    "temperature": [latest_temperature] * len(future_dates),
    **{f"pm_2_5_lag_{lag}": [latest_pm25] * len(future_dates) for lag in lags}
})

# เพิ่มฟีเจอร์เชิงเวลา
future_data["hour"] = future_data["datetime"].dt.hour
future_data["dayofweek"] = future_data["datetime"].dt.dayofweek
future_data["month"] = future_data["datetime"].dt.month
future_data["season"] = (future_data["month"] % 12 + 3) // 3
future_data["is_daytime"] = future_data["hour"].apply(lambda x: 1 if 6 <= x <= 18 else 0)

# Recursive forecasting
for i in range(len(future_data)):
    # ทำนายค่า PM 2.5
    pred = rf_model.predict(future_data.iloc[i:i+1].drop(columns=["datetime"]))
    predicted_value = pred[0]
    
    # อัปเดต Lag Features สำหรับแถวถัดไป
    for lag in lags:
        if i + lag < len(future_data):
            future_data.at[i + lag, f"pm_2_5_lag_{lag}"] = predicted_value

# ทำนายค่า PM 2.5 สำหรับข้อมูลอนาคต
future_predictions = rf_model.predict(future_data.drop(columns=["datetime"]))

# เพิ่มคอลัมน์การทำนายลงใน future_data
future_data["predicted_pm_2_5"] = future_predictions

# แสดงผลลัพธ์
print(future_data[["datetime", "predicted_pm_2_5"]])

# บันทึกผลลัพธ์เป็นไฟล์ CSV
# future_data.to_csv("D:/Term_Project/pm_2.5/predicted_pm25.csv", index=False)
# print("บันทึกผลลัพธ์การพยากรณ์สำเร็จ!")

Mean Squared Error: 14.271946821054875
R^2 Score: 0.79969993673472
               datetime  predicted_pm_2_5
0   2025-01-28 00:00:00         18.154091
1   2025-01-28 01:00:00         17.979270
2   2025-01-28 02:00:00         17.780799
3   2025-01-28 03:00:00         17.671116
4   2025-01-28 04:00:00         17.850767
..                  ...               ...
163 2025-02-03 19:00:00         17.737858
164 2025-02-03 20:00:00         18.624463
165 2025-02-03 21:00:00         17.772405
166 2025-02-03 22:00:00         17.627840
167 2025-02-03 23:00:00         18.008698

[168 rows x 2 columns]
